In [2]:
import fasttext
import pandas as pd
import re

In [3]:
filepath = 'data.xlsx'

data = pd.read_excel(filepath)

In [4]:
data.head(5)

,Numéro d'application,Date d'application,Numero de publication,date de publication,CPC,IPC,claim,description
0,17167919,2017-04-25,3264223,2018-01-03,"['B23K26-361', 'H04M1-0266', 'B23K26-402', 'G0...","['C08J7-04', 'H04M1-02', 'G06F1-16', 'B23K26-3...","<!-- EPO <DP n=""18""> --><claim id=""c-en-0001"" ...","<!-- EPO <DP n=""1""> --><p id=""p0001"" num=""0001..."
1,17166775,2017-04-18,3235458,2017-10-25,"['A61B17-68', 'A61B17-888', 'A61B17-8685', 'A6...","['A61B17-88', 'A61B17-68', 'A61B17-86']","<!-- EPO <DP n=""20""> --><claim id=""c-en-0001"" ...","<!-- EPO <DP n=""1""> --><heading id=""h0001"">FIE..."
2,17166069,2017-04-11,3249094,2017-11-29,"['D06F58-206', 'D06F58-20']","['D06F58-20', 'D06F58-02']","<!-- EPO <DP n=""22""> --><claim id=""c-en-0001"" ...","<!-- EPO <DP n=""1""> --><heading id=""h0001""><b>..."
3,18771363,2018-03-05,3605724,2020-02-05,"['H01P1-2138', 'H01P5-107', 'H01P1-2088', 'H01...","['H01P3-12', 'H01P1-213', 'H01P5-107']","<!-- EPO <DP n=""62""> --><claim id=""c-en-0001"" ...","<!-- EPO <DP n=""1""> --><heading id=""h0001"">Tec..."
4,19881950,2019-11-01,3713245,2020-09-23,"['H04N21-4345', 'H04N21-4821', 'H04N21-4316']","['H04N21-482', 'H04N21-262']","<!-- EPO <DP n=""14""> --><claim id=""c-en-0001"" ...","<!-- EPO <DP n=""1""> --><p id=""p0001"" num=""0001..."


In [5]:
import re
import ast
from collections import Counter

def split_code(l):

    # Cette fonction prend une liste de codes alphanumériques et retourne une nouvelle liste
    # contenant les sections les plus fréquentes à chaque position, tout en respectant
    # la validité des sélections précédentes pour maintenir la correspondance avec les codes originaux.
    
    l = ast.literal_eval(l)
    splitted_code_list = []
    splitted_code = []

    # Diviser chaque code et séparer les lettres et les chiffres
    for elt in l:
        elt = elt.split('-')
        letters = list(filter(None, re.split(r'[0-9]', elt[0])))
        digits = list(filter(None, re.split(r'[A-Z]', elt[0])))
        code = [sub[item] for item in range(len(digits)) for sub in [letters, digits]]
        code.append(elt[1])
        splitted_code.append(code)

    # Déterminer les sections les plus prédominantes tout en maintenant la validité
    result = []
    filtered_codes = splitted_code

    for i in range(len(splitted_code[0])):
        # Extraire les éléments ièmes des codes filtrés
        ith_elements = [code[i] for code in filtered_codes]
        
        # Trouver l'élément le plus commun
        most_common, _ = Counter(ith_elements).most_common(1)[0]
        
        # Ajouter l'élément le plus commun au résultat
        result.append(most_common)
        
        # Filtrer les codes pour la prochaine itération
        filtered_codes = [code for code in filtered_codes if code[i] == most_common]

    # We add a the concatenated code to the result
    #result.append('-'.join(result[0:3]) + '-' + result[3])

    return result

# Exemple d'utilisation
res = split_code("['B23K26-361', 'H04C1-0266', 'B23K26-402', 'M04C1-0256', 'U04C1-0246']")
print(res)

['B', '23', 'K', '26', '361']


In [6]:
cpc_cleaned = data['CPC'].apply(split_code).apply(pd.Series)
cpc_cleaned.columns = ['CPC_section','CPC_class','CPC_subclass','CPC_group','CPC_subgroup']

# We add the columns to the original dataframe
data = pd.concat([data, cpc_cleaned], axis=1)

data.drop(['Numéro d\'application', 'Date d\'application','Numero de publication', 'date de publication'], axis=1, inplace=True)

data.head()

,CPC,IPC,claim,description,CPC_section,CPC_class,CPC_subclass,CPC_group,CPC_subgroup
0,"['B23K26-361', 'H04M1-0266', 'B23K26-402', 'G0...","['C08J7-04', 'H04M1-02', 'G06F1-16', 'B23K26-3...","<!-- EPO <DP n=""18""> --><claim id=""c-en-0001"" ...","<!-- EPO <DP n=""1""> --><p id=""p0001"" num=""0001...",B,23,K,26,361
1,"['A61B17-68', 'A61B17-888', 'A61B17-8685', 'A6...","['A61B17-88', 'A61B17-68', 'A61B17-86']","<!-- EPO <DP n=""20""> --><claim id=""c-en-0001"" ...","<!-- EPO <DP n=""1""> --><heading id=""h0001"">FIE...",A,61,B,17,68
2,"['D06F58-206', 'D06F58-20']","['D06F58-20', 'D06F58-02']","<!-- EPO <DP n=""22""> --><claim id=""c-en-0001"" ...","<!-- EPO <DP n=""1""> --><heading id=""h0001""><b>...",D,06,F,58,206
3,"['H01P1-2138', 'H01P5-107', 'H01P1-2088', 'H01...","['H01P3-12', 'H01P1-213', 'H01P5-107']","<!-- EPO <DP n=""62""> --><claim id=""c-en-0001"" ...","<!-- EPO <DP n=""1""> --><heading id=""h0001"">Tec...",H,01,P,1,2138
4,"['H04N21-4345', 'H04N21-4821', 'H04N21-4316']","['H04N21-482', 'H04N21-262']","<!-- EPO <DP n=""14""> --><claim id=""c-en-0001"" ...","<!-- EPO <DP n=""1""> --><p id=""p0001"" num=""0001...",H,04,N,21,4345


In [7]:
import re
import nltk
import string
from nltk.corpus import stopwords

nltk.download('stopwords')

def clean_text(text):
    # Remove HTML tags
    text = re.sub('<.*?>', ' ', text)
    
    # Remove specific patterns (e.g., '-->')
    text = re.sub('-->', '', text)
    
    # Remove punctuation
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator)
    
    # Convert to lowercase
    text = text.lower()
    
    # Remove numbers
    text = re.sub(r'[0-9]', '', text)
    
    # Remove extra whitespace
    text = " ".join(text.split())
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    text = ' '.join(filtered_words)
    
    return text


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/youssouf/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
data['description'] = data['description'] + ' ' + data['claim']
data['description'] = data['description'].apply(clean_text)

data.drop(['claim'], axis=1, inplace=True)

In [9]:
data.head(1)

,CPC,IPC,description,CPC_section,CPC_class,CPC_subclass,CPC_group,CPC_subgroup
0,"['B23K26-361', 'H04M1-0266', 'B23K26-402', 'G0...","['C08J7-04', 'H04M1-02', 'G06F1-16', 'B23K26-3...",present invention relates cover window display...,B,23,K,26,361


In [10]:
import os
import pandas as pd

def generate_fasttext_files(data):
    """
    Main function to generate FastText format files for hierarchical classification.
    """
    # Create a main directory to store the FastText files
    output_dir = "fasttext_files"
    os.makedirs(output_dir, exist_ok=True)

    # Step 1: Generate FastText file for CPC section
    section_dir = os.path.join(output_dir, "sections")
    os.makedirs(section_dir, exist_ok=True)
    create_fasttext_file(data, "CPC_section", os.path.join(section_dir, "fasttext_section.txt"))

    # Get unique CPC sections
    unique_sections = data["CPC_section"].unique()

    # Step 2: For each section, generate FastText file for CPC class
    for section in unique_sections:
        filtered_data = filter_data(data, "CPC_section", section)
        class_dir = os.path.join(section_dir, f"section_{section}")
        os.makedirs(class_dir, exist_ok=True)
        output_file = os.path.join(class_dir, f"fasttext_section_{section}_class.txt")
        create_fasttext_file(filtered_data, "CPC_class", output_file)

        # Get unique CPC classes within this section
        unique_classes = filtered_data["CPC_class"].unique()

        # Step 3: For each class, generate FastText file for CPC subclass
        for cpc_class in unique_classes:
            filtered_data_class = filter_data(filtered_data, "CPC_class", cpc_class)
            subclass_dir = os.path.join(class_dir, f"class_{cpc_class}")
            os.makedirs(subclass_dir, exist_ok=True)
            output_file = os.path.join(subclass_dir, f"fasttext_section_{section}_class_{cpc_class}_subclass.txt")
            create_fasttext_file(filtered_data_class, "CPC_subclass", output_file)

            # Get unique CPC subclasses within this class
            unique_subclasses = filtered_data_class["CPC_subclass"].unique()

            # Step 4: For each subclass, generate FastText file for CPC group
            for subclass in unique_subclasses:
                filtered_data_subclass = filter_data(filtered_data_class, "CPC_subclass", subclass)
                group_dir = os.path.join(subclass_dir, f"subclass_{subclass}")
                os.makedirs(group_dir, exist_ok=True)
                output_file = os.path.join(group_dir, f"fasttext_section_{section}_class_{cpc_class}_subclass_{subclass}_group.txt")
                create_fasttext_file(filtered_data_subclass, "CPC_group", output_file)

                # Get unique CPC groups within this subclass
                unique_groups = filtered_data_subclass["CPC_group"].unique()

                # Step 5: For each group, generate FastText file for CPC subgroup
                for group in unique_groups:
                    filtered_data_group = filter_data(filtered_data_subclass, "CPC_group", group)
                    subgroup_dir = os.path.join(group_dir, f"group_{group}")
                    os.makedirs(subgroup_dir, exist_ok=True)
                    output_file = os.path.join(subgroup_dir, f"fasttext_section_{section}_class_{cpc_class}_subclass_{subclass}_group_{group}_subgroup.txt")
                    create_fasttext_file(filtered_data_group, "CPC_subgroup", output_file)

def create_fasttext_file(data, target_column, output_file):
    """
    Creates a FastText formatted file for the given target column and saves it to the specified output file.
    """
    with open(output_file, 'w') as file:
        for index, row in data.iterrows():
            text = row['description']
            label = row[target_column]
            file.write(f"__label__{label} {text}\n")

def filter_data(data, filter_column, filter_value):
    """
    Filters the dataframe based on the specified column and value.
    """
    return data[data[filter_column] == filter_value]




In [11]:

generate_fasttext_files(data)  # Call the function to generate FastText files


In [12]:
import os
import fasttext

def train_fasttext_models(base_dir):
    """
    Train FastText models for each hierarchical level.
    """
    # Step 1: Train model for CPC section
    section_file = os.path.join(base_dir, "sections", "fasttext_section.txt")
    section_model = fasttext.train_supervised(input=section_file)
    section_model.save_model(os.path.join(base_dir, "section_model.bin"))

    # Get subfolders for sections
    section_dirs = [d for d in os.listdir(os.path.join(base_dir, "sections")) if os.path.isdir(os.path.join(base_dir, "sections", d))]

    # Step 2: Train model for each CPC class
    for section_dir in section_dirs:
        class_file = os.path.join(base_dir, "sections", section_dir, f"fasttext_{section_dir}_class.txt")
        class_model = fasttext.train_supervised(input=class_file)
        class_model.save_model(os.path.join(base_dir, "sections", section_dir, "class_model.bin"))

        # Get subfolders for classes
        class_dirs = [d for d in os.listdir(os.path.join(base_dir, "sections", section_dir)) if os.path.isdir(os.path.join(base_dir, "sections", section_dir, d))]

        # Step 3: Train model for each CPC subclass
        for class_dir in class_dirs:
            subclass_file = os.path.join(base_dir, "sections", section_dir, class_dir, f"fasttext_{section_dir}_{class_dir}_subclass.txt")
            subclass_model = fasttext.train_supervised(input=subclass_file)
            subclass_model.save_model(os.path.join(base_dir, "sections", section_dir, class_dir, "subclass_model.bin"))

            # Get subfolders for subclasses
            subclass_dirs = [d for d in os.listdir(os.path.join(base_dir, "sections", section_dir, class_dir)) if os.path.isdir(os.path.join(base_dir, "sections", section_dir, class_dir, d))]

            # Step 4: Train model for each CPC group
            for subclass_dir in subclass_dirs:
                group_file = os.path.join(base_dir, "sections", section_dir, class_dir, subclass_dir, f"fasttext_{section_dir}_{class_dir}_{subclass_dir}_group.txt")
                group_model = fasttext.train_supervised(input=group_file)
                group_model.save_model(os.path.join(base_dir, "sections", section_dir, class_dir, subclass_dir, "group_model.bin"))

                # Get subfolders for groups
                group_dirs = [d for d in os.listdir(os.path.join(base_dir, "sections", section_dir, class_dir, subclass_dir)) if os.path.isdir(os.path.join(base_dir, "sections", section_dir, class_dir, subclass_dir, d))]

                # Step 5: Train model for each CPC subgroup
                for group_dir in group_dirs:
                    subgroup_file = os.path.join(base_dir, "sections", section_dir, class_dir, subclass_dir, group_dir, f"fasttext_{section_dir}_{class_dir}_{subclass_dir}_{group_dir}_subgroup.txt")
                    subgroup_model = fasttext.train_supervised(input=subgroup_file)
                    subgroup_model.save_model(os.path.join(base_dir, "sections", section_dir, class_dir, subclass_dir, group_dir, "subgroup_model.bin"))

# Base directory where the FastText files are stored
base_dir = "fasttext_files"


In [13]:

# Train the FastText models
train_fasttext_models(base_dir)



Read 147M words
Number of words:  454919
Number of labels: 9
Progress: 100.0% words/sec/thread: 7914506 lr:  0.000000 avg.loss:  0.857600 ETA:   0h 0m 0s
Read 13M words
Number of words:  51071
Number of labels: 18
Progress: 100.0% words/sec/thread: 7203539 lr:  0.000000 avg.loss:  2.462111 ETA:   0h 0m 0s
Read 0M words
Number of words:  8309
Number of labels: 5
Progress: 100.0% words/sec/thread: 1246700 lr:  0.000000 avg.loss:  1.613351 ETA:   0h 0m 0s
Read 0M words
Number of words:  2388
Number of labels: 6
Progress: 100.0% words/sec/thread:  130767 lr:  0.000000 avg.loss:  1.795673 ETA:   0h 0m 0s
Read 0M words
Number of words:  272
Number of labels: 1
Progress: 100.0% words/sec/thread:   13917 lr:  0.000000 avg.loss:  0.000000 ETA:   0h 0m 0s
Read 0M words
Number of words:  952
Number of labels: 2
Progress: 100.0% words/sec/thread: 3770992 lr:  0.000000 avg.loss:  0.695104 ETA:   0h 0m 0s
Read 0M words
Number of words:  694
Number of labels: 2
Progress: 100.0% words/sec/thread:   34

In [14]:
data['description'][1]


'field invention present disclosure relates implantable screws particularly disclosure relates implantable screws configured provide compression upon bone screw implanted background invention broken bone must carefully stabilized supported strong enough handle bodys weight movement last century physicians relied casts splints support stabilize bone outside body advent sterile surgical procedures reduced risk infection allowing doctors internally set stabilize fractured bones surgical procedure set fracture bone fragments first repositioned reduced normal alignment held together special implants plates screws nails wires screws used internal fixation often type implant although screw simple device different designs based type fracture screw used screws come different sizes use bones different sizes screws used alone hold fracture well plates rods nails bone heals screws may either left place removed many instances desired inserted screw provide compression bone joint fracture line reduc

In [15]:
data[['CPC_section','CPC_class','CPC_subclass','CPC_group','CPC_subgroup']]

,CPC_section,CPC_class,CPC_subclass,CPC_group,CPC_subgroup
0,B,23,K,26,361
1,A,61,B,17,68
2,D,06,F,58,206
3,H,01,P,1,2138
4,H,04,N,21,4345
...,...,...,...,...,...
49995,A,61,K,2800,95
49996,B,60,R,7,02
49997,C,07,K,2317,21
49998,G,07,F,11,62
